In [23]:
import pandas as pd

# Load your data frame (assuming it's named 'data')
df = pd.read_csv('C:/Users/Dell/Documents/Business Analytics Year 4/Applied Predictive Analytics DATA4001 2022-23/final4.csv')

In [24]:
import pandas as pd

# Load the data into a DataFrame
df = pd.read_csv('C:/Users/Dell/Documents/Business Analytics Year 4/Applied Predictive Analytics DATA4001 2022-23/final4.csv')


# Add a new column called "Position" and label each row based on the value in the "Pos" column
df['Position'] = df['Pos'].apply(lambda x: 'Point Guard' if x == 'PG' else ('Small Forward' if x == 'SF' else ('Power Forward' if x == 'PF' else ('Center' if x == 'C' else 'Shooting Gaurd'))))

# Define the formulas for each position
formulas = {
    'Point Guard': lambda data: ((0.2 * data['PTS'] + 0.4 * data['AST'] + 0.05 * data['TRB']) / data['MP'] + 0.3 * (data['FG%'] + data['3P%'] + data['2P%']) / 3) + (0.3 * (data['BLK'] + data['STL'] + data['DRB'] - data['PF']) / data['MP']),
    'Small Forward': lambda data: ((0.5 * data['PTS'] + 0.2 * data['AST'] + 0.1 * data['TRB']) / data['MP'] + 0.2 * (data['FG%'] + 0.7 * data['3P%'] + 0.3 * data['2P%']) / 3 ) + (0.4 * (data['BLK'] + data['STL'] + data['DRB'] - data['PF']) / data['MP']),
    'Power Forward': lambda data: ((0.5 * data['PTS'] + 0.05 * data['AST'] + 0.25 * data['TRB']) / data['MP'] + 0.2 * (data['FG%'] + data['3P%'] + data['2P%']) / 3 ) + (0.4 * (data['BLK'] + data['STL'] + data['DRB'] - data['PF']) / data['MP']),
    'Center': lambda data: ((0.4 * data['PTS'] + 0.05 * data['AST'] + 0.35 * data['TRB']) / data['MP'] + 0.2 * (data['FG%'] + 0.1 * data['3P%'] + 0.9 * data['2P%']) / 3) + (0.7 * (data['BLK'] + data['STL'] + data['DRB'] - data['PF']) /data['MP']),
    'Shooting Gaurd': lambda data: ((0.4 * data['PTS'] + 0.3 * data['AST'] + 0.1 * data['TRB']) / data['MP'] + 0.2 * (data['FG%'] + 0.3*data['3P%'] + 0.7*data['2P%']) / 3) + (0.4 * (data['BLK'] + data['STL'] + data['DRB'] - data['PF']) / data['MP'])}

# Initialize the Overall column to 0.0
df['Overall'] = 0.0

# Loop through each position and calculate the Overall rating for each player
for position in df['Position'].unique():
    position_data = df.loc[df['Position'] == position]
    for season in position_data['Season'].unique():
        season_data = position_data.loc[(position_data['Season'] == season) & (position_data['G'] >= 20)]
        season_data['Overall'] = formulas[position](season_data)
        df.loc[(df['Season'] == season) & (df['G'] >= 20) & (df['Position'] == position), 'Overall'] = season_data['Overall']

# Sort the data by Overall rating
df = df.sort_values(by='Overall', ascending=False)
print(df.head(10))

C:\Users\Dell\AppData\Local\Temp/ipykernel_23844/3968862427.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  season_data['Overall'] = formulas[position](season_data)


      Rk                 Player Pos  Age   Tm   G  GS    MP    FG   FGA  ...  \
449   13  Giannis Antetokounmpo  PF   25  MIL  63  63  30.4  10.9  19.7  ...   
221   12  Giannis Antetokounmpo  PF   27  MIL  67  67  32.9  10.3  18.6  ...   
373  155            Joel Embiid   C   24  PHI  64  64  33.7   9.1  18.7  ...   
440  162            Joel Embiid   C   27  PHI  68  68  33.8   9.8  19.6  ...   
544  146            Joel Embiid   C   26  PHI  51  51  31.1   9.0  17.6  ...   
703   18  Giannis Antetokounmpo  PF   24  MIL  72  72  32.8  10.0  17.3  ...   
390  126          Anthony Davis   C   25  NOP  56  56  33.0   9.5  18.3  ...   
531   12  Giannis Antetokounmpo  PF   26  MIL  61  61  33.0  10.3  18.0  ...   
407  100          Anthony Davis   C   23  NOP  75  75  36.1  10.3  20.3  ...   
442  146            Joel Embiid   C   25  PHI  51  51  29.5   7.5  15.7  ...   

     TOV   PF   PTS   Season    MVP  Player salary  Salary Cap  %_of_cap  \
449  3.7  3.1  29.5  2019-20   True       2

In [30]:
top3 = df.groupby(['Season', 'Pos']).apply(lambda x: x.nlargest(3, 'Overall'))
top3


Rk            Player Pos  Age   Tm   G  GS    MP   FG  \
Season  Pos                                                                
1997-98 C   37    331    David Robinson   C   32  SAS  73  73  33.7  7.5   
            15    123     Patrick Ewing   C   35  NYK  26  26  32.6  7.8   
            1640  345   Arvydas Sabonis   C   33  POR  73  73  32.0  5.6   
        PF  738   236       Karl Malone  PF   34  UTA  81  81  37.4  9.6   
            2452  110        Tim Duncan  PF   21  SAS  82  82  39.1  8.6   
...               ...               ...  ..  ...  ...  ..  ..   ...  ...   
2021-22 SF  493   526      Jayson Tatum  SF   23  BOS  76  76  35.9  9.3   
            290   189       Paul George  SF   31  LAC  31  31  34.7  8.6   
        SG  303    59      Devin Booker  SG   25  PHO  68  68  34.5  9.7   
            674   383  Donovan Mitchell  SG   25  UTA  67  67  33.8  9.2   
            61    218      James Harden  SG   32  PHI  21  21  37.7  5.5   

                   FGA  ...  TOV   PF   PTS   Season    MVP  Player salary  \
Season  Pos             ...                                                  
1997-98 C   37    14.6  ...  2.8  2.8  21.6  1997-98  False       12397440   
            15    15.5  ...  3.0  2.8  20.8  1997-98  False       20500000   
            1640  11.3  ...  2.6  3.7  16.0  1997-98  False        3080000   
        PF  738   18.2  ...  3.0  2.9  27.0  1997-98  False        5118578   
            2452  15.7  ...  3.4  3.1  21.1  1997-98  False        2967840   
...                ...  ...  ...  ...   ...      ...    ...            ...   
2021-22 SF  493   20.6  ...  2.9  2.3  26.9  2021-22  False       28103500   
            290   20.5  ...  4.1  2.4  24.3  2021-22  False       39344970   
        SG  303   20.9  ...  2.4  2.6  26.8  2021-22  False       31650600   
            674   20.5  ...  3.0  2.4  25.9  2021-22  False       28103500   
            61    13.6  ...  3.4  2.3  21.0  2021-22  False       44310840   

                  Salary Cap  %_of_cap        Position   Overall  
Season  Pos                                                       
1997-98 C   37      38846163  0.319142          Center  0.603100  
            15      53974881  0.379806          Center  0.600003  
            1640    27786719  0.110844          Center  0.516094  
        PF  738     28505442  0.179565   Power Forward  0.604965  
            2452    38846163  0.076400   Power Forward  0.511830  
...                      ...       ...             ...       ...  
2021-22 SF  493    136557646  0.205800   Small Forward  0.547683  
            290    168378382  0.233670   Small Forward  0.534042  
        SG  303    136476474  0.231912  Shooting Gaurd  0.467651  
            674    149760719  0.187656  Shooting Gaurd  0.459769  
            61     148922969  0.297542  Shooting Gaurd  0.440149  

[375 rows x 37 columns]

In [26]:
df.to_csv('Formulas.csv', index=False)